## Imports

In [ ]:
import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb

In this notebook, we'll define a test suite to "CheckList" different sentiment models.The idea is to check different linguistic capabilities using tests derived from software engineering principles. This notebook is heavily based on the 'sentimnent' notebook in the original CheckList re 

In [ ]:
#TODO -> what is an editor and suite?
editor = checklist.editor.Editor()
suite = TestSuite()

## Capability: Vocabulary

### MFT (Minimal Functionality Test)

In [6]:
#Adding relevant movie nouns 
movie_noun = ['movie', 'actor', 'director', 'producer', 'crew', 'script', 'production house']
editor.add_lexicon('air_noun', movie_noun)

In [9]:
pos_adj = ['good', 'great', 'excellent', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'incredible', 'exceptional', 'awesome', 'perfect', 'fun', 'happy', 'adorable', 'brilliant', 'exciting', 'sweet', 'wonderful']
neg_adj = ['awful', 'bad', 'horrible', 'weird', 'rough', 'lousy', 'unhappy', 'average', 'difficult', 'poor', 'sad', 'frustrating', 'hard', 'lame', 'nasty', 'annoying', 'boring', 'creepy', 'dreadful', 'ridiculous', 'terrible', 'ugly', 'unpleasant']

editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )

In [11]:
## Create MFTs & add to test suite
test = MFT(pos_adj, labels=1)
suite.add(test, 'single positive words', 'Vocabulary', '')

test = MFT(neg_adj, labels=0)
suite.add(test, 'single negative words', 'Vocabulary', '')

### Save 

In [14]:
### Save suite as pkl file
path = './sentiment_suite.pkl'
suite.save(path)